### Hybrid Solver (CSP + LLM)

Our Hybrid Solver, is a solver that combines Constraint Satisfaction Problem (CSP) techniques with a Language Learning Model (LLM) to efficiently solve Wordle puzzles. Here's how it works:

1. Language Agent (LLM Component)
    - Uses OpenAI's model (GPT 4o) to make strategic word selections
    - Analyzes patterns and letter distributions in remaining candidates
    - Calculates information gain for potential guesses
    - Provides explanations for word choices

2. CSP Component:
    - The Constraint Satisfaction Problem component handles the logical filtering of the word list.
    - Maintains constraints using the WordleConstraints class (green, yellow, grey letters)
    - Tracks minimum letter counts to handle duplicate letters correctly,
    - Efficiently filters the word list after each guess via the filter_valid_words function,
    - Ensures all candidates are valid according to game rules,

3. Solving Process in Detail:,
    - **Initialization**: Starts with a complete dictionary of valid words,
    - **First Move Optimization**: For large dictionaries, uses \"crane\" as a statistically efficient first guess,
    - **Iterative Guessing**:,
        * The LLM agent suggests optimal next words through function-calling capabilities,
        * Feedback is collected comparing the guess against the target word
        * CSP filters the dictionary to maintain only valid candidates
        * Process repeats until solution is found or max attempts reached

4. Example of Solving Flow:
    1. Start with complete dictionary (e.g., 2,315 words),
    2. First guess \"crane\" → get feedback (e.g., 🟨⬛🟩⬛⬛),
    3. CSP filters dictionary to words matching constraints (e.g., 42 words)
    4. LLM analyzes remaining candidates, suggests optimal word through function calling
    5. Process repeats until solution is found or we reach 6 guesses

## How does the LLM make Strategic Decisions
1. Strategic Decision Making:
    - **Small Candidate Lists** (≤3 words):
        * Performs direct information gain calculation for each candidate
        * Selects the word that would provide maximum information
        - **Larger Candidate Lists**:
          * LLM performs deeper analysis using multiple function calls:
            - evaluate_information_gain: Calculates expected entropy reduction
            - analyze_letter_patterns: Examines letter frequencies and positional patterns
            - explain_choice: Provides reasoning behind the selection, which can be seen in the frontend

2. Information Gain Calculation:
    - Computes initial entropy of remaining word list
    - For each potential guess, simulates all possible feedback patterns
    - Groups remaining candidates by feedback pattern
    - Calculates expected entropy after guess
    - Information gain = initial entropy - expected entropy

Example:
- Imagine we have 8 possible words remaining: ["crate", "grade","crane", "trace", "space", "brace", "grace", "track"]

If the LLM wants to calculate the information gain for guessing "crane":
- Calculate initial entropy:
    * 8 possible words = log₂(8) = 3 bits of uncertainty
- Simulate feedback for "crane" against each possible target:
    * For "crate": 🟩🟩⬛🟨⬛ (feedback pattern A)
    * For "grade": ⬛🟩⬛🟨⬛ (feedback pattern B)
    * For "crane": 🟩🟩🟩🟩🟩 (feedback pattern C)
    * For "trace": 🟨🟩⬛🟨⬛ (feedback pattern D)
    * For "space": ⬛⬛🟩⬛⬛ (feedback pattern E)
    * For "brace": ⬛🟩🟩⬛⬛ (feedback pattern F)
    * For "grace": ⬛🟩🟩⬛⬛ (feedback pattern F again)
    * For "track": ⬛⬛🟩⬛🟨 (feedback pattern G)
- Group words by feedback pattern:
    * Pattern A: ["crate"] (1 word)
    * Pattern B: ["grade"] (1 word)
    * Pattern C: ["crane"] (1 word)
    * Pattern D: ["trace"] (1 word)
    * Pattern E: ["space"] (1 word)
    * Pattern F: ["brace", "grace"] (2 words)
    * Pattern G: ["track"] (1 word)

- Calculate expected entropy after guess:
    * P(A) = 1/8, entropy = log₂(1) = 0
    * P(B) = 1/8, entropy = log₂(1) = 0
    * P(C) = 1/8, entropy = log₂(1) = 0
    * P(D) = 1/8, entropy = log₂(1) = 0
    * P(E) = 1/8, entropy = log₂(1) = 0
    * P(F) = 2/8, entropy = log₂(2) = 1
    * P(G) = 1/8, entropy = log₂(1) = 0

Expected entropy = (1/8 × 0) + (1/8 × 0) + (1/8 × 0) + (1/8 × 0) + (1/8 × 0) + (2/8 × 1) + (1/8 × 0) = 0.25

Information gain = Initial entropy - Expected entropy = 3 - 0.25 = 2.75 bits

- This means "crane" gives us 2.75 bits of information, which is very good (close to the maximum possible 3 bits). After guessing "crane", we'll likely narrow down to just 1 or 2 possible words.
